# IMPORTS

In [25]:
from datetime import datetime

from clientes.analisis_fuensalida import analisis_fuensalida_params as params
from read_data_base.data_collectors import SQLServerCollector
from read_data_base.softland_querys import QueryTipoDocumentos,QueryAuxiliares,QueryTodosMovimientos,QueryTodosComprobantes

from analisis_cuentas.informe_cuentas_softland import remove_initial_movements

import pandas as pd

# GET DATA

In [12]:
input_date = datetime.strptime("30/09/2021","%d/%m/%Y")

collector = SQLServerCollector( params.server,
                                params.port,
                                params.database,
                                params.username,
                                params.password)

docs_query = QueryTipoDocumentos()
auxiliares_query = QueryAuxiliares()
movimientos_query = QueryTodosMovimientos(params.reembolsos)
comprobantes_query = QueryTodosComprobantes()

#Connect to database
collector.connect()

#Read Doc Types
all_docs_df = collector.get_data(docs_query)

#Read Auxiliares
auxiliares = collector.get_data(auxiliares_query)

#Read comprobantes
comprobantes = collector.get_data(comprobantes_query)

#Read Movements
movimientos = collector.get_data(movimientos_query)

# TESTING AREA

In [16]:
# clean_movements = remove_initial_movements(movimientos)
clean_movements = movimientos[movimientos.MovGlosa.str.contains('Gasto|gasto|reembolso|Reembolso')]
# clean_movements = movimientos[movimientos.TipDocCb != '00']
# filtered_movements = clean_movements.groupby(['CodAux']).filter(lambda group: group.Debe.sum()-group.Haber.sum()!=0).sort_values(['CodAux','CpbAno','CpbNum'])
# # clean_movements.set_index(['CodAux','CpbAno','CpbNum'])
clean_movements.to_excel('movimientors_reembolsos.xlsx')

In [21]:
comprobantes[(comprobantes.CpbAno == '2017')&((comprobantes.CpbNum == '00002070')|(comprobantes.CpbNum == '00002081'))].to_excel('cpb.xlsx')

In [26]:
pd.merge(
        movimientos,
        comprobantes,
        how="left",
        left_on=['CpbAno','CpbNum'],
        right_on=['CpbAno','CpbNum'],
        suffixes=("", "_")).to_excel('movimientors_reembolsos.xlsx')

,CpbAno,CpbNum,MovNum,AreaCod,PctCod,CpbFec,CpbMes,CvCod,VendCod,UbicCod,...,Proceso,Usuario,CpbNormaIFRS_,CpbNormaTrib_,CpbAnoRev,CpbNumRev,SistemaMod,ProcesoMod,FechaUlMod,TipoLog
0,2016,00010054,0.0,000,1-1-05-001-005,2016-10-03,10,000,0000,000,...,Comprobante,abr,S,S,0000,00000000,CW,Comprobante,2017-03-22 15:17:03.860,U
1,2016,00010054,1.0,000,1-1-05-001-005,2016-10-03,10,000,0000,000,...,Comprobante,abr,S,S,0000,00000000,CW,Comprobante,2017-03-22 15:17:03.860,U
2,2016,00010054,2.0,000,1-1-05-001-005,2016-10-03,10,000,0000,000,...,Comprobante,abr,S,S,0000,00000000,CW,Comprobante,2017-03-22 15:17:03.860,U
3,2016,00010054,3.0,000,1-1-05-001-005,2016-10-03,10,000,0000,000,...,Comprobante,abr,S,S,0000,00000000,CW,Comprobante,2017-03-22 15:17:03.860,U
4,2016,00010054,4.0,000,1-1-05-001-005,2016-10-03,10,000,0000,000,...,Comprobante,abr,S,S,0000,00000000,CW,Comprobante,2017-03-22 15:17:03.860,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4434,2021,00000352,23.0,000,1-1-05-001-005,2021-10-31,10,000,0000,000,...,Captura de Transacciones,jam,S,S,0000,00000000,CW,Comprobante,2021-11-11 17:34:05.240,U
4435,2021,00000352,24.0,000,1-1-05-001-005,2021-10-31,10,000,0000,000,...,Captura de Transacciones,jam,S,S,0000,00000000,CW,Comprobante,2021-11-11 17:34:05.240,U
4436,2021,00000352,25.0,000,1-1-05-001-005,2021-10-31,10,000,0000,000,...,Captura de Transacciones,jam,S,S,0000,00000000,CW,Comprobante,2021-11-11 17:34:05.240,U
4437,2021,00000352,26.0,000,1-1-05-001-005,2021-10-31,10,000,0000,000,...,Captura de Transacciones,jam,S,S,0000,00000000,CW,Comprobante,2021-11-11 17:34:05.240,U
